# Job Market Analysis — Step-by-step

This notebook is created for learning: it walks through **data loading**, **cleaning**, **EDA**, **time-series aggregation**, **seasonal decomposition**, **forecasting (SARIMAX & RandomForest)**, and **exporting cleaned files** ready for Power BI.

**How to use**: open this notebook in JupyterLab / Jupyter Notebook, run cells top-to-bottom. Explanations are included before each code cell.

Files used/produced in this environment:
- Input: `/mnt/data/ai_job_dataset.csv`
- Outputs (saved by the notebook): `/mnt/data/ai_job_dataset_cleaned.csv` and `/mnt/data/ai_job_timeseries_weekly.csv`

---


In [ ]:
# Imports
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

plt.rcParams['figure.figsize'] = (10,4)


## 1) Load dataset
Replace the path below if your file is elsewhere. We detect columns and display a quick sample.

In [ ]:
DATA_PATH = '/mnt/data/ai_job_dataset.csv'
df = pd.read_csv(DATA_PATH)
df.shape
# show top 5 rows
df.head()

## 2) Clean column names
Lowercase, replace spaces with underscores and remove special characters. This simplifies later code.

In [ ]:
def clean_column_names(df):
    df = df.copy()
    df.columns = (df.columns.str.strip()
                            .str.lower()
                            .str.replace(' ', '_')
                            .str.replace('[^0-9a-zA-Z_]', '', regex=True))
    return df

df = clean_column_names(df)
print('Columns after cleaning:')
print(df.columns.tolist())


## 3) Detect date & salary columns (best-effort)
We look for common hints like 'date' or 'salary' and try to parse date-like columns.

In [ ]:
def guess_date_column(df):
    hints = ['date','posted','created','published','post','listed']
    for col in df.columns:
        if any(h in col for h in hints):
            parsed = pd.to_datetime(df[col], errors='coerce')
            if parsed.notna().mean() > 0.3:
                return col
    # fallback: highest parseable fraction
    best = None; best_frac=0
    for col in df.columns:
        parsed = pd.to_datetime(df[col], errors='coerce')
        frac = parsed.notna().mean()
        if frac>best_frac:
            best_frac=frac; best=col
    return best if best_frac>0.5 else None

def guess_salary_column(df):
    hints = ['salary','ctc','package','pay','compensation']
    for col in df.columns:
        if any(h in col for h in hints):
            return col
    return None

DATE_COL = guess_date_column(df)
SALARY_COL = guess_salary_column(df)
print('Guessed date col:', DATE_COL)
print('Guessed salary col:', SALARY_COL)
if DATE_COL:
    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors='coerce')
    print('Non-null dates:', df[DATE_COL].notna().mean())


## 4) Parse salary column (best-effort)
This function converts common salary formats into numeric values (handles k, ranges, LPA/lakh, currency symbols).

In [ ]:
import math

def parse_salary_text(s):
    if pd.isna(s):
        return np.nan
    if isinstance(s,(int,float)) and not math.isnan(s):
        return float(s)
    text = str(s).lower().replace(',','').replace('$','').replace('inr','').replace('rs','').replace('₹','')
    if 'lpa' in text or 'lakh' in text or 'lac' in text:
        nums = [float(''.join(ch for ch in t if (ch.isdigit() or ch=='.'))) for t in text.replace('-', ' ').split() if any(c.isdigit() for c in t)]
        if nums:
            return np.mean(nums) * 100000
    if 'k' in text:
        parts = text.split('-')
        nums = []
        for p in parts:
            n = ''.join(ch for ch in p if (ch.isdigit() or ch=='.'))
            if n:
                nums.append(float(n))
        if nums:
            return np.mean(nums) * 1000
    if '-' in text or 'to' in text:
        sep = '-' if '-' in text else 'to'
        parts = [p.strip() for p in text.split(sep)]
        nums = []
        for p in parts:
            n = ''.join(ch for ch in p if (ch.isdigit() or ch=='.'))
            if n:
                nums.append(float(n))
        if nums:
            return np.mean(nums)
    num = ''.join(ch for ch in text if (ch.isdigit() or ch=='.'))
    try:
        return float(num) if num else np.nan
    except:
        return np.nan

if SALARY_COL:
    df['_salary_parsed'] = df[SALARY_COL].apply(parse_salary_text)
    df['_salary_parsed'].describe()
else:
    print('No salary column detected.')


## 5) Basic text cleaning (job title, company, location)
Lowercase and strip whitespace to standardize categories so aggregations are cleaner.

In [ ]:
for c in ['job_title','job','title','company','location','company_name','company_location']:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.replace('\n',' ').str.replace('\r',' ').str.lower()

# Show top job titles
for c in ['job_title','title','job']:
    if c in df.columns:
        print('\nTop job titles:')
        print(df[c].value_counts().head(10))
        break

if 'company' in df.columns:
    print('\nTop companies:')
    print(df['company'].value_counts().head(10))

if 'location' in df.columns:
    print('\nTop locations:')
    print(df['location'].value_counts().head(10))


## 6) Missing values & decisions
We display columns with missing values and decide simple imputation/drop strategies (explain choices).

In [ ]:
miss = df.isna().sum().sort_values(ascending=False)
miss[miss>0]


## 7) Time-series aggregation
Aggregate posting counts weekly and monthly from the detected date column. These series will be used for decomposition & forecasting.

In [ ]:
if DATE_COL:
    df_dates = df.dropna(subset=[DATE_COL]).set_index(DATE_COL)
    ts_weekly = df_dates.resample('W').size().rename('job_count')
    ts_monthly = df_dates.resample('M').size().rename('job_count')
    display(ts_weekly.head())
    plt.plot(ts_weekly.index, ts_weekly.values); plt.title('Weekly job counts'); plt.show()
else:
    print('No date column detected; create one before running time-series steps.')


## 8) Seasonal decomposition
If the weekly series is long enough, decompose into trend/seasonality/residuals.

In [ ]:
if 'ts_weekly' in globals() and len(ts_weekly)>=24:
    tsw = ts_weekly.asfreq('W').fillna(0)
    res = seasonal_decompose(tsw, period=52, model='additive', extrapolate_trend='freq')
    res.plot(); plt.show()
else:
    print('Weekly series too short for decomposition (need ~24+ weeks).')


## 9) Forecasting
We attempt two approaches:
- SARIMAX (statistical model)
- RandomForest with lag features (machine learning model)

We evaluate with RMSE and MAE.

In [ ]:
def evaluate(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    return {'rmse':rmse,'mae':mae}

# SARIMAX
if 'ts_weekly' in globals() and len(ts_weekly)>=24:
    tsw = ts_weekly.asfreq('W').fillna(0)
    n_test = min(12, max(4, int(len(tsw)*0.2)))
    train = tsw.iloc[:-n_test]; test = tsw.iloc[-n_test:]
    model = SARIMAX(train, order=(1,1,1), seasonal_order=(0,0,0,0), enforce_stationarity=False, enforce_invertibility=False)
    sarima_res = model.fit(disp=False)
    pred = sarima_res.get_forecast(steps=len(test))
    pred_mean = pred.predicted_mean
    plt.plot(train.index, train.values, label='train'); plt.plot(test.index, test.values, label='test'); plt.plot(pred_mean.index, pred_mean.values, label='pred'); plt.legend(); plt.show()
    print('SARIMAX eval:', evaluate(test.values, pred_mean.values))
else:
    print('Not enough data for SARIMAX')

# RandomForest with lag features
if 'ts_weekly' in globals() and len(ts_weekly)>=24:
    tsw = ts_weekly.asfreq('W').fillna(0)
    df_lag = tsw.to_frame('y')
    n_lags = 12
    for lag in range(1,n_lags+1):
        df_lag[f'lag_{lag}'] = df_lag['y'].shift(lag)
    df_lag = df_lag.dropna()
    n_test = min(12, max(4, int(len(df_lag)*0.2)))
    train = df_lag.iloc[:-n_test]; test = df_lag.iloc[-n_test:]
    X_train = train.drop(columns=['y']); y_train = train['y']
    X_test = test.drop(columns=['y']); y_test = test['y']
    rf = RandomForestRegressor(n_estimators=200, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    plt.plot(y_test.index, y_test.values, label='actual'); plt.plot(y_test.index, y_pred, label='pred'); plt.legend(); plt.show()
    print('RF eval:', evaluate(y_test.values, y_pred))
else:
    print('Not enough data for ML forecasting')


## 10) Save cleaned datasets for Power BI
Save the cleaned dataset and weekly timeseries CSV for direct import into Power BI.

In [ ]:
CLEANED_PATH = '/mnt/data/ai_job_dataset_cleaned.ipynb_placeholder.csv'
TS_PATH = '/mnt/data/ai_job_timeseries_weekly.csv'
# Save cleaned dataframe (CSV)
df.to_csv('/mnt/data/ai_job_dataset_cleaned.csv', index=False)
print('Saved cleaned CSV to /mnt/data/ai_job_dataset_cleaned.csv')
# Save weekly timeseries if exists
if 'ts_weekly' in globals():
    ts_weekly.reset_index().to_csv(TS_PATH, index=False)
    print('Saved weekly timeseries to', TS_PATH)


---
## Notes: Power BI tips
1. Import the cleaned CSV via **Get data → Text/CSV**.
2. In Power Query set `posting_date` type to *Date* and Close & Apply.
3. Use **Line chart** with date on the axis (choose Month/Week) and job count as value.
4. Use the Analytics pane → Forecast to add an interactive forecast for presentations.

---

End of notebook. Happy learning!